# 梧桐杯 AI 安全告警智能研判系统 - V5 最终版（冲击 99.8%+）

## 🎯 V5 核心优化

1. ⭐⭐⭐⭐⭐ **外部数据集整合**：CSIC 2010 (36k+)，预期 +0.5-1.0%
2. ⭐⭐⭐⭐⭐ **SMOTE 过采样**：解决类别不平衡，预期 +0.8-1.2%
3. ⭐⭐⭐⭐ **XGBoost 深度优化**：500 树 + 深度 12，预期 +0.3-0.5%
4. 🐛 **修复警告**：移除 scale_pos_weight 参数
5. ⚡ **智能聚合器**：大数据集（>50k）自动跳过聚合以节省时间
6. ✨ **10 折交叉验证**：更稳定的性能评估

## 📋 训练流程

1. ✅ 克隆 GitHub 仓库
2. ✅ 安装依赖（含 imbalanced-learn）
3. ✅ 检查 GPU
4. ✅ 检查数据
5. ✅ 数据预处理
6. ✅ 数据增强
7. ✅ **下载并整合 CSIC 2010 数据集（V5 核心）**
8. ✅ **训练 V5 模型（SMOTE + 外部数据集 + 优化超参数）**
9. ✅ 查看结果
10. ✅ 下载模型

## 🎯 目标

- **准确率目标**: 99.8%+
- **V3 基线**: 98.36%
- **V4 基线**: 99.2-99.5%
- **预期提升**: +1.4-1.6%
- **训练时间**: 约 40-50 分钟
- **模型版本**: v5.0.0

## ⚙️ 运行前准备

1. 确保运行时类型设置为 **GPU**（Runtime > Change runtime type > GPU）
2. 按顺序执行每个单元格

---

In [ ]:
# 1. 克隆 GitHub 仓库
!git clone https://github.com/alltobebetter/WuTong.git
%cd WuTong

# 查看项目结构
!ls -la

In [ ]:
# 2. 安装依赖（含 imbalanced-learn）
print("📦 安装依赖包...")
!pip install -q -r requirements.txt

print("\n✅ 依赖安装完成！")

# 验证关键包
import xgboost as xgb
import catboost as cb
import lightgbm as lgb
import pandas as pd
from imblearn.over_sampling import SMOTE

print(f"XGBoost 版本: {xgb.__version__}")
print(f"CatBoost 版本: {cb.__version__}")
print(f"LightGBM 版本: {lgb.__version__}")
print(f"Pandas 版本: {pd.__version__}")
print(f"✅ SMOTE 已安装")

In [ ]:
# 3. 检查 GPU 可用性
import torch

if torch.cuda.is_available():
    print(f"✅ GPU 可用: {torch.cuda.get_device_name(0)}")
    print(f"   显存: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
else:
    print("⚠️ GPU 不可用，将使用 CPU 训练（速度较慢）")
    print("   建议: Runtime > Change runtime type > GPU")

In [ ]:
# 4. 检查数据集
import pandas as pd
from pathlib import Path

data_files = list(Path('data').rglob('*.xlsx'))
print(f"找到 {len(data_files)} 个数据文件:")
for f in data_files:
    print(f"  - {f}")

if data_files:
    df = pd.read_excel(data_files[0])
    print(f"\n数据集大小: {len(df)} 条")
    print(f"\n攻击类型分布:")
    print(df.iloc[:, -1].value_counts())
else:
    print("\n⚠️ 未找到数据文件！")

In [ ]:
# 5. 数据预处理
print("🔄 开始数据预处理...")

import glob
excel_files = glob.glob('data/**/*.xlsx', recursive=True)

if excel_files:
    data_file = excel_files[0]
    print(f"使用数据文件: {data_file}")
    !python scripts/ingest.py "{data_file}"
    print("\n✅ 数据预处理完成！")

    parquet_files = glob.glob('data/staging/*.parquet')
    print(f"\n生成的 parquet 文件: {parquet_files}")
else:
    print("❌ 未找到 Excel 数据文件！")

In [ ]:
# 6. 数据增强
print("="*60)
print("🚀 数据增强开始")
print("="*60)
print("\n策略:")
print("  - SQL 注入: 关键词替换、注释变换、空格编码")
print("  - XSS 攻击: 标签大小写、事件处理器、编码变换")
print("  - 命令注入: 分隔符变换、命令组合")
print("  - URL 路径: 大小写、编码、路径分隔符")
print("\n目标: 11,000 → 30,000+ 条 (2.7x)\n")

!python scripts/augment_data.py --target-size 30000 --ratio 2.5

print("\n" + "="*60)
print("✅ 数据增强完成！")
print("="*60)

# 查看增强后的数据
import pandas as pd
import glob

augmented_files = glob.glob('data/staging/*augmented*.parquet')
if augmented_files:
    df_aug = pd.read_parquet(augmented_files[0])
    print(f"\n📊 增强后统计:")
    print(f"  总数据量: {len(df_aug)} 条")
    print(f"  增强倍数: {len(df_aug) / 11000:.2f}x")
    print(f"\n类别分布:")
    print(df_aug['attack_type'].value_counts())

In [ ]:
# 7. 下载并整合 CSIC 2010 数据集（V5 核心）
print("="*60)
print("🚀 下载并整合 CSIC 2010 数据集")
print("="*60)
print("\n数据集信息:")
print("  - 来源: Spanish Research National Council (CSIC)")
print("  - 规模: 36,000+ 正常流量 + 25,000+ 攻击流量")
print("  - 类型: SQL 注入、XSS、缓冲区溢出、目录遍历等")
print("  - 预期提升: +0.5-1.0%\n")

!python scripts/integrate_csic2010.py

print("\n" + "="*60)
print("✅ CSIC 2010 整合完成！")
print("="*60)

# 查看外部数据集
import pandas as pd
import glob

external_files = glob.glob('data/external/*.parquet')
if external_files:
    df_ext = pd.read_parquet(external_files[0])
    print(f"\n📊 外部数据集统计:")
    print(f"  总数据量: {len(df_ext)} 条")
    print(f"\n类别分布:")
    print(df_ext['attack_type'].value_counts())
    
    # 计算总数据量
    augmented_files = glob.glob('data/staging/*augmented*.parquet')
    if augmented_files:
        df_aug = pd.read_parquet(augmented_files[0])
        total = len(df_aug) + len(df_ext)
        print(f"\n📈 总数据量: {total} 条 ({total / 11000:.1f}x)")
else:
    print("\n⚠️ 未找到外部数据集")

In [ ]:
# 8. 训练 V5 模型（外部数据集 + SMOTE + 优化超参数）
print("="*60)
print("🚀 V5 模型训练开始（外部数据集 + SOTA 优化）")
print("="*60)
print("\n配置:")
print("  - 模型: XGBoost + CatBoost + LightGBM")
print("  - 集成方式: Stacking (优化 meta-learner)")
print("  - 交叉验证: 10 折")
print("  - ⭐ 外部数据集: CSIC 2010 (36k+)")
print("  - ⭐ SMOTE 过采样: 是（解决类别不平衡）")
print("  - ⭐ XGBoost 优化: 500 树 + 深度 12（修复警告）")
print("  - 预计时间: 40-50 分钟\n")

import subprocess
cmd = [
    "python", "-u", "scripts/train_v5.py",
    "--version", "v5.0.0",
    "--cv-splits", "10"
]
proc = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1)
for line in proc.stdout:
    print(line, end='')
ret = proc.wait()
if ret != 0:
    raise RuntimeError(f"训练失败，退出码: {ret}")

print("\n" + "="*60)
print("✅ V5 训练完成！")
print("="*60)

In [ ]:
# 9. 查看 V5 训练结果
import json
from pathlib import Path

manifest_path = Path('models/v5.0.0/manifest.json')

if manifest_path.exists():
    with open(manifest_path, 'r', encoding='utf-8') as f:
        manifest = json.load(f)

    print("="*70)
    print("📊 V5 训练结果（外部数据集 + SOTA 优化）")
    print("="*70)
    print(f"\n版本: {manifest['version']}")
    print(f"训练时间: {manifest['trained_at']}")
    print(f"数据量: {manifest['data_rows']} 条")
    print(f"外部数据集: {'是' if manifest.get('external_data') else '否'}")
    if manifest.get('external_data'):
        print(f"外部数据量: {manifest.get('external_data_rows', 0)} 条")
    print(f"特征数: {len(manifest['feature_list'])} 个")
    print(f"类别数: {len(manifest['classes'])} 类")

    config = manifest.get('training_config', {})
    print(f"\n训练配置:")
    print(f"  - 交叉验证: {config.get('n_cv_splits', 'N/A')} 折")
    print(f"  - 集成方式: {config.get('use_stacking', False) and 'Stacking' or 'Voting'}")
    print(f"  - SMOTE 过采样: {config.get('use_smote', False) and '是' or '否'}")

    print("\n" + "="*70)
    print("🎯 模型性能")
    print("="*70)

    metrics = manifest['metrics']

    # 单模型性能
    for model_name in ['xgboost', 'catboost', 'lightgbm']:
        if model_name in metrics:
            m = metrics[model_name]
            print(f"\n{model_name.upper()}:")
            print(f"  测试准确率: {m['test_accuracy']:.4f} ({m['test_accuracy']*100:.2f}%)")
            print(f"  测试 F1: {m['test_f1']:.4f}")
            if m.get('cv_accuracy'):
                print(f"  CV 准确率: {m['cv_accuracy']:.4f} (±{m.get('cv_std', 0):.4f})")

    # 集成模型性能
    if 'ensemble' in metrics:
        e = metrics['ensemble']
        print(f"\n{'='*70}")
        print(f"🏆 V5 集成模型（最终模型）")
        print(f"{'='*70}")
        print(f"  准确率: {e['test_accuracy']:.4f} ({e['test_accuracy']*100:.2f}%)")
        print(f"  F1 分数: {e['test_f1']:.4f}")
        print(f"  集成方式: {e.get('ensemble_type', 'N/A')}")

        # 与 V3/V4 对比
        v3_acc = 0.9836  # V3 的准确率
        v4_acc = 0.9920  # V4 预期准确率（假设）
        improvement_v3 = (e['test_accuracy'] - v3_acc) * 100
        improvement_v4 = (e['test_accuracy'] - v4_acc) * 100
        print(f"\n  📈 相比 V3 提升: {improvement_v3:+.2f}%")
        print(f"  📈 相比 V4 提升: {improvement_v4:+.2f}%")

        if e['test_accuracy'] >= 0.998:
            print(f"\n  🎉🎉🎉 恭喜！达到 99.8% 目标！")
        elif e['test_accuracy'] >= 0.995:
            print(f"\n  🎉🎉 恭喜！达到 99.5% 目标！")
        elif e['test_accuracy'] >= 0.99:
            print(f"\n  🎉 恭喜！达到 99% 目标！")
        elif e['test_accuracy'] >= 0.985:
            print(f"\n  ✨ 非常接近 99%，表现优秀！")

    print("\n" + "="*70)

    # 详细分类报告
    report_path = Path('models/v5.0.0/classification_report.txt')
    if report_path.exists():
        print("\n📋 详细分类报告:")
        print("="*70)
        with open(report_path, 'r', encoding='utf-8') as f:
            print(f.read())
else:
    print("❌ 未找到 V5 训练结果文件！")
    print("请先运行训练单元格")

In [ ]:
# 10. 打包并下载模型
import shutil
from pathlib import Path

model_dir = Path('models/v5.0.0')
if model_dir.exists():
    print("📦 打包模型文件...")
    shutil.make_archive('models_v5.0.0', 'zip', 'models', 'v5.0.0')
    print("✅ 打包完成: models_v5.0.0.zip")
    
    # 在 Colab 中下载
    from google.colab import files
    files.download('models_v5.0.0.zip')
    print("\n✅ 下载已开始！")
else:
    print("❌ 模型目录不存在！")